### ResNet50 Keras baseline model

This notebook takes you through some important steps in building a deep convnet in Keras for multilabel classification of brain CT scans. 

In [1]:
!pip install git+https://github.com/qubvel/efficientnet

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-p29bhz4m
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-p29bhz4m
  Created wheel for efficientnet: filename=efficientnet-1.0.0b3-cp36-none-any.whl size=17713 sha256=c1e35206799357e102788a4479c4aa7e09fd0581bdd91decad2dc5e8e8139022
  Stored in directory: /tmp/pip-ephem-wheel-cache-ngfebla1/wheels/64/60/2e/30ebaa76ed1626e86bfb0cc0579b737fdb7d9ff8cb9522663a
Successfully built efficientnet


In [2]:
import numpy as np
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import collections
from tqdm import tqdm_notebook as tqdm
from datetime import datetime

from math import ceil, floor
import cv2

import tensorflow as tf
import keras

import sys

from keras.applications.mobilenet import MobileNet
from efficientnet.keras import EfficientNetB0

from sklearn.model_selection import ShuffleSplit

Using TensorFlow backend.


### 1. Helper functions

For windowing the input images (thanks to fellow competitors' notebooks\*), and to normalize the pixel values between -1 and 1.

\* Source: https://www.kaggle.com/omission/eda-view-dicom-images-with-correct-windowing

In [3]:
def _get_first_of_dicom_field_as_int(x):
    if type(x) == pydicom.multival.MultiValue:
        return int(x[0])
    else:
        return int(x)

def _get_windowing(data):
    dicom_fields = [data.WindowCenter, data.WindowWidth, data.RescaleSlope, data.RescaleIntercept]
    return [_get_first_of_dicom_field_as_int(x) for x in dicom_fields]


def _window_image(img, window_center, window_width, slope, intercept):
    img = (img * slope + intercept)
    img_min = window_center - window_width//2
    img_max = window_center + window_width//2
    img[img<img_min] = img_min
    img[img>img_max] = img_max
    return img 

def _normalize(img):
    if img.max() == img.min():
        return np.zeros(img.shape)
    return (img - img.min())/(img.max() - img.min())


def _read(path, desired_size=(224, 224)):
    """Will be used in DataGenerator"""
    
    dcm = pydicom.dcmread(path)

    window_params = _get_windowing(dcm) # (center, width, slope, intercept)

    try:
        # dcm.pixel_array might be corrupt (one case so far)
        img = _window_image(dcm.pixel_array, *window_params)
    except:
        img = np.zeros(desired_size)

    img = _normalize(img)

    if desired_size != (512, 512):
        # resize image
        img = cv2.resize(img, desired_size, interpolation=cv2.INTER_LINEAR)

    return img[:,:,np.newaxis]


### 2. Data generators

You could make this in to just one DataGenerator, but I kept it as two separate DataGenerators (one for training and one for predicting). It inherits from keras.utils.Sequence object and thus should be safe for multiprocessing.


In [4]:
class TrainDataGenerator(keras.utils.Sequence):

    def __init__(self, list_IDs, labels, batch_size=1, img_size=(512, 512), 
                 img_dir='../input/data/stage_1_train_images/', *args, **kwargs):

        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indices]
        X, Y = self.__data_generation(list_IDs_temp)
        return X, Y

    def on_epoch_end(self):
        self.indices = np.arange(len(self.list_IDs))
        np.random.shuffle(self.indices)

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.img_size, 1))
        Y = np.empty((self.batch_size, 6), dtype=np.float32)
        
        for i, ID in enumerate(list_IDs_temp):
            X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
            Y[i,] = self.labels.loc[ID].values
        
        return X, Y
    
    
class TestDataGenerator(keras.utils.Sequence):

    def __init__(self, list_IDs, labels, batch_size=1, img_size=(512, 512), 
                 img_dir='../input/data/stage_1_test_images/', *args, **kwargs):

        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indices]
        X = self.__data_generation(list_IDs_temp)
        return X

    def on_epoch_end(self):
        self.indices = np.arange(len(self.list_IDs))

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.img_size, 1))
        
        for i, ID in enumerate(list_IDs_temp):
            X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
        
        return X

### 3. Model

Basically a combination of three models, which are sequentially concatenated. <br> 

* The initial layer, which will transform/map input image of shape (\_, \_, 1) to another "image" of shape (\_, \_, 3).

* The new input image is then passed through ResNet50 (which I named "engine"). ResNet50 could be replaced by any of the available architectures in keras_application.

* Finally, the output from ResNet50 goes through average pooling followed by a dense output layer.

In [5]:
def _initial_layer(input_dims):
    inputs = keras.layers.Input(input_dims)
    
    x = keras.layers.Conv2D(filters=3, kernel_size=(1, 1), strides=(1, 1), name="initial_conv2d")(inputs)
    x = keras.layers.BatchNormalization(axis=3, epsilon=1.001e-5, name='initial_bn')(x)
    x = keras.layers.Activation('relu', name='initial_relu')(x)
    
    return keras.models.Model(inputs, x)

class MyDeepModel:
    
    def __init__(self, engine, input_dims, batch_size=5, learning_rate=1e-3, 
                 decay_rate=1.0, decay_steps=1, weights="imagenet", verbose=1):
        
        self.engine = engine
        self.input_dims = input_dims
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.decay_rate = decay_rate
        self.decay_steps = decay_steps
        self.weights = weights
        self.verbose = verbose
        self._build()

    def _build(self):
        
        initial_layer = _initial_layer((*self.input_dims, 1))
    
        engine = self.engine(include_top=False, weights=self.weights, input_shape=(*self.input_dims, 3),
#                             backend = keras.backend, layers = keras.layers,
#                             models = keras.models, utils = keras.utils)
                            )

        x = engine(initial_layer.output)

        x = keras.layers.GlobalAveragePooling2D(name='avg_pool')(x)

        out = keras.layers.Dense(6, activation="sigmoid", name='dense_output')(x)

        self.model = keras.models.Model(inputs=initial_layer.input, outputs=out)

        self.model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(0.0))
    
    
    def fit(self, df, train_idx, img_dir, global_epoch):
        self.model.fit_generator(
            TrainDataGenerator(
                df.iloc[train_idx].index, 
                df.iloc[train_idx], 
                self.batch_size, 
                self.input_dims, 
                img_dir
            ),
            verbose=self.verbose,
            use_multiprocessing=True,
            workers=4,
            callbacks=[
                keras.callbacks.LearningRateScheduler(
                    lambda epoch: self.learning_rate * pow(self.decay_rate, floor(global_epoch / self.decay_steps))
                )
            ]
        )
    
    def predict(self, df, test_idx, img_dir):
        predictions = \
          self.model.predict_generator(
            TestDataGenerator(
                df.iloc[test_idx].index, 
                None, 
                self.batch_size, 
                self.input_dims, 
                img_dir
            ),
            verbose=1,
            use_multiprocessing=True,
            workers=4
        )

        return predictions[:df.iloc[test_idx].shape[0]]
    
    def save(self, path):
        self.model.save_weights(path)
    
    def load(self, path):
        self.model.load_weights(path)

In [6]:
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
def handle_block_names(stage, block):
    name_base = 'stage{}_unit{}_'.format(stage + 1, block + 1)
    conv_name = name_base + 'conv'
    bn_name = name_base + 'bn'
    relu_name = name_base + 'relu'
    sc_name = name_base + 'sc'
    return conv_name, bn_name, relu_name, sc_name


def basic_identity_block(filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """

    def layer(input_tensor):
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = Activation('relu', name=relu_name + '1')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), name=conv_name + '1', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = Activation('relu', name=relu_name + '2')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), name=conv_name + '2', **conv_params)(x)

        x = Add()([x, input_tensor])
        return x

    return layer


def basic_conv_block(filters, stage, block, strides=(2, 2)):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """

    def layer(input_tensor):
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = Activation('relu', name=relu_name + '1')(x)
        shortcut = x
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), strides=strides, name=conv_name + '1', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = Activation('relu', name=relu_name + '2')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), name=conv_name + '2', **conv_params)(x)

        shortcut = Conv2D(filters, (1, 1), name=sc_name, strides=strides, **conv_params)(shortcut)
        x = Add()([x, shortcut])
        return x

    return layer


def usual_conv_block(filters, stage, block, strides=(2, 2)):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """

    def layer(input_tensor):
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = Activation('relu', name=relu_name + '1')(x)
        shortcut = x
        x = Conv2D(filters, (1, 1), name=conv_name + '1', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = Activation('relu', name=relu_name + '2')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), strides=strides, name=conv_name + '2', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '3', **bn_params)(x)
        x = Activation('relu', name=relu_name + '3')(x)
        x = Conv2D(filters*4, (1, 1), name=conv_name + '3', **conv_params)(x)

        shortcut = Conv2D(filters*4, (1, 1), name=sc_name, strides=strides, **conv_params)(shortcut)
        x = Add()([x, shortcut])
        return x

    return layer


def usual_identity_block(filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """

    def layer(input_tensor):
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = Activation('relu', name=relu_name + '1')(x)
        x = Conv2D(filters, (1, 1), name=conv_name + '1', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = Activation('relu', name=relu_name + '2')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), name=conv_name + '2', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '3', **bn_params)(x)
        x = Activation('relu', name=relu_name + '3')(x)
        x = Conv2D(filters*4, (1, 1), name=conv_name + '3', **conv_params)(x)

        x = Add()([x, input_tensor])
        return x

    return layer

In [7]:
def get_conv_params(**params):
    default_conv_params = {
        'kernel_initializer': 'glorot_uniform',
        'use_bias': False,
        'padding': 'valid',
    }
    default_conv_params.update(params)
    return default_conv_params

def get_bn_params(**params):
    default_bn_params = {
        'axis': 3,
        'momentum': 0.99,
        'epsilon': 2e-5,
        'center': True,
        'scale': True,
    }
    default_bn_params.update(params)
    return default_bn_params

import keras.backend as K
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import GlobalAveragePooling2D
from keras.layers import ZeroPadding2D
from keras.layers import Dense
from keras.models import Model
from keras.engine import get_source_inputs

import keras
from distutils.version import StrictVersion

if StrictVersion(keras.__version__) < StrictVersion('2.2.0'):
    from keras.applications.imagenet_utils import _obtain_input_shape
else:
    from keras_applications.imagenet_utils import _obtain_input_shape

def build_resnet(
     repetitions=(2, 2, 2, 2),
     include_top=True,
     input_tensor=None,
     input_shape=None,
     classes=1000,
     block_type='usual'):

    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=101,
                                      data_format='channels_last',
                                      require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape, name='data')
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    
    # get parameters for model layers
    no_scale_bn_params = get_bn_params(scale=False)
    bn_params = get_bn_params()
    conv_params = get_conv_params()
    init_filters = 64

    if block_type == 'basic':
        conv_block = basic_conv_block
        identity_block = basic_identity_block
    else:
        conv_block = usual_conv_block
        identity_block = usual_identity_block
    
    # resnet bottom
    x = BatchNormalization(name='bn_data', **no_scale_bn_params)(img_input)
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(init_filters, (7, 7), strides=(2, 2), name='conv0', **conv_params)(x)
    x = BatchNormalization(name='bn0', **bn_params)(x)
    x = Activation('relu', name='relu0')(x)
    x = ZeroPadding2D(padding=(1, 1))(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='valid', name='pooling0')(x)
    
    # resnet body
    for stage, rep in enumerate(repetitions):
        for block in range(rep):
            
            filters = init_filters * (2**stage)
            
            # first block of first stage without strides because we have maxpooling before
            if block == 0 and stage == 0:
                x = conv_block(filters, stage, block, strides=(1, 1))(x)
                
            elif block == 0:
                x = conv_block(filters, stage, block, strides=(2, 2))(x)
                
            else:
                x = identity_block(filters, stage, block)(x)
                
    x = BatchNormalization(name='bn1', **bn_params)(x)
    x = Activation('relu', name='relu1')(x)

    # resnet top
    if include_top:
        x = GlobalAveragePooling2D(name='pool1')(x)
        x = Dense(classes, name='fc1')(x)
        x = Activation('softmax', name='softmax')(x)

    # Ensure that the model takes into account any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
        
    # Create model.
    model = Model(inputs, x)

    return model

weights_collection = [
    # ResNet34
    {
        'model': 'resnet34',
        'dataset': 'imagenet',
        'classes': 1000,
        'include_top': True,
        'url': 'https://github.com/qubvel/classification_models/releases/download/0.0.1/resnet34_imagenet_1000.h5',
        'name': 'resnet34_imagenet_1000.h5',
        'md5': '2ac8277412f65e5d047f255bcbd10383',
    },

    {
        'model': 'resnet34',
        'dataset': 'imagenet',
        'classes': 1000,
        'include_top': False,
        'url': 'https://github.com/qubvel/classification_models/releases/download/0.0.1/resnet34_imagenet_1000_no_top.h5',
        'name': 'resnet34_imagenet_1000_no_top.h5',
        'md5': '8caaa0ad39d927cb8ba5385bf945d582',
    },
]
def ResNet34(input_shape, input_tensor=None, weights=None, classes=1000, include_top=True):
    model = build_resnet(input_tensor=input_tensor,
                         input_shape=input_shape,
                         repetitions=(3, 4, 6, 3),
                         classes=classes,
                         include_top=include_top,
                         block_type='basic')
    model.name = 'resnet34'

    if weights:
        load_model_weights(weights_collection, model, weights, classes, include_top)
    return model

from keras.utils import get_file


def find_weights(weights_collection, model_name, dataset, include_top):
    w = list(filter(lambda x: x['model'] == model_name, weights_collection))
    w = list(filter(lambda x: x['dataset'] == dataset, w))
    w = list(filter(lambda x: x['include_top'] == include_top, w))
    return w


def load_model_weights(weights_collection, model, dataset, classes, include_top):
    weights = find_weights(weights_collection, model.name, dataset, include_top)

    if weights:
        weights = weights[0]

        if include_top and weights['classes'] != classes:
            raise ValueError('If using `weights` and `include_top`'
                             ' as true, `classes` should be {}'.format(weights['classes']))

        weights_path = get_file(weights['name'],
                                weights['url'],
                                cache_subdir='models',
                                md5_hash=weights['md5'])

        model.load_weights(weights_path)

    else:
        raise ValueError('There is no weights for such configuration: ' +
                         'model = {}, dataset = {}, '.format(model.name, dataset) +
                         'classes = {}, include_top = {}.'.format(classes, include_top))

### 4. Read csv files


In [8]:
def read_testset(filename="../input/rsna-intracranial-hemorrhage-detection/stage_1_sample_submission.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    
    return df

def read_trainset(filename="../input/rsna-intracranial-hemorrhage-detection/stage_1_train.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    
    duplicates_to_remove = [
        1598538, 1598539, 1598540, 1598541, 1598542, 1598543,
        312468,  312469,  312470,  312471,  312472,  312473,
        2708700, 2708701, 2708702, 2708703, 2708704, 2708705,
        3032994, 3032995, 3032996, 3032997, 3032998, 3032999
    ]
    
    df = df.drop(index=duplicates_to_remove)
    df = df.reset_index(drop=True)
    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    
    return df

    
test_df1 = read_testset()
test_df2 = read_testset()
df = read_trainset()

In [9]:
df.head(3)

Label                                                          \
Diagnosis      any epidural intraparenchymal intraventricular subarachnoid   
Image                                                                        
ID_000039fa0     0        0                0                0            0   
ID_00005679d     0        0                0                0            0   
ID_00008ce3c     0        0                0                0            0   

                       
Diagnosis    subdural  
Image                  
ID_000039fa0        0  
ID_00005679d        0  
ID_00008ce3c        0

In [10]:
test_df1.head(3)

Label                                                          \
Diagnosis      any epidural intraparenchymal intraventricular subarachnoid   
Image                                                                        
ID_000012eaf   0.5      0.5              0.5              0.5          0.5   
ID_0000ca2f6   0.5      0.5              0.5              0.5          0.5   
ID_000259ccf   0.5      0.5              0.5              0.5          0.5   

                       
Diagnosis    subdural  
Image                  
ID_000012eaf      0.5  
ID_0000ca2f6      0.5  
ID_000259ccf      0.5

### 5. Train model and predict

*Using train, validation and test set* <br>

Training for 3 epochs with Adam optimizer. The learning rate starts at 0.001, with a slight decay (0.75) each epoch. The predictions are then \[exponentially weighted\] averaged over all 3 epochs. Same goes for the test set submission later.

In [11]:
_TEST_IMAGES = '../input/rsna-intracranial-hemorrhage-detection/stage_1_test_images/'
_TRAIN_IMAGES = '../input/rsna-intracranial-hemorrhage-detection/stage_1_train_images/'

def run_any(model, df, train_idx, valid_idx, test_df, epochs):
    
    valid_predictions = []
    test_predictions = []
    for global_epoch in range(epochs):

        model.fit(df, train_idx, _TRAIN_IMAGES, global_epoch)
        
        test_predictions.append(model.predict(test_df, range(test_df.shape[0]), _TEST_IMAGES))
        val_pred = model.predict(df, valid_idx, _TRAIN_IMAGES)
        valid_predictions.append(val_pred)
        
        print("weighted validation loss: %.4f" %
              weighted_loss_metric(df.iloc[valid_idx].values, 
                                   np.average(valid_predictions, axis=0, 
                                              weights=[2**i for i in range(len(valid_predictions))]))
             )
        
        print("epoch validation loss: %.4f" %
              weighted_loss_metric(df.iloc[valid_idx].values, val_pred)
             )
        
    final_pred = model.predict(test_df, range(test_df.shape[0]), _TEST_IMAGES)
    return test_predictions, valid_predictions, final_pred



def weighted_loss_metric(trues, preds, weights=[0.2, 0.1, 0.1, 0.1, 0.1, 0.1], clip_value=1e-7):
    """this is probably not correct, but works OK. Feel free to give feedback."""
    preds = np.clip(preds, clip_value, 1-clip_value)
    loss_subtypes = trues * np.log(preds) + (1 - trues) * np.log(1 - preds)
    loss_weighted = np.average(loss_subtypes, axis=1, weights=weights)
    return - loss_weighted.mean()

# train set (90%) and validation set (10%)
ss = ShuffleSplit(n_splits=5, test_size=0.1, random_state=42).split(df.index)

# will just do one fold
train_idx, valid_idx = next(ss)

# obtain model
model = MyDeepModel(engine=EfficientNetB0, input_dims=(224, 224), batch_size=32, learning_rate=1e-3, 
                    decay_rate=0.75, decay_steps=1, weights="imagenet", verbose=1)

16809984/16804768 [==============================] - 1s 0us/step


In [12]:
# run 3 epochs and obtain test + validation predictions
test_preds, _, final_pred = run_any(model, df, train_idx, valid_idx, test_df1, 3)

Epoch 1/1
2108/2108 [==============================] - 355s 168ms/step
weighted validation loss: 0.1328
epoch validation loss: 0.1328
Epoch 1/1
2108/2108 [==============================] - 358s 170ms/step
weighted validation loss: 0.0905
epoch validation loss: 0.0817
Epoch 1/1
2455/2455 [==============================] - 401s 163ms/step


### 6. Submit test predictions

In [13]:
test_df1.iloc[:, :] = np.average(test_preds, axis=0, weights=[2**i for i in range(len(test_preds))])

test_df1 = test_df1.stack().reset_index()

test_df1.insert(loc=0, column='ID', value=test_df1['Image'].astype(str) + "_" + test_df1['Diagnosis'])

test_df1 = test_df1.drop(["Image", "Diagnosis"], axis=1)

test_df1

,ID,Label
0,ID_000012eaf_any,0.055565
1,ID_000012eaf_epidural,0.001494
2,ID_000012eaf_intraparenchymal,0.017800
3,ID_000012eaf_intraventricular,0.004454
4,ID_000012eaf_subarachnoid,0.005425
...,...,...
471265,ID_ffffcbff8_epidural,0.000339
471266,ID_ffffcbff8_intraparenchymal,0.006955
471267,ID_ffffcbff8_intraventricular,0.016373
471268,ID_ffffcbff8_subarachnoid,0.003124


In [14]:
test_df1.to_csv('eff_b0_weighted_submission.csv', index=False)

In [15]:
test_df2.iloc[:, :] = final_pred

test_df2 = test_df2.stack().reset_index()

test_df2.insert(loc=0, column='ID', value=test_df2['Image'].astype(str) + "_" + test_df2['Diagnosis'])

test_df2 = test_df2.drop(["Image", "Diagnosis"], axis=1)

test_df2

,ID,Label
0,ID_000012eaf_any,0.016680
1,ID_000012eaf_epidural,0.000151
2,ID_000012eaf_intraparenchymal,0.009127
3,ID_000012eaf_intraventricular,0.004001
4,ID_000012eaf_subarachnoid,0.003007
...,...,...
471265,ID_ffffcbff8_epidural,0.000184
471266,ID_ffffcbff8_intraparenchymal,0.006091
471267,ID_ffffcbff8_intraventricular,0.003362
471268,ID_ffffcbff8_subarachnoid,0.001343


In [16]:
test_df2.to_csv('eff_b0_final_epoch_submission.csv', index=False)

### 7. Improvements

Some improvements that could be made:<br>
* Image augmentation (which can be put in `_read()`)
* Different learning rate and learning rate schedule
* Increased input size
* Train longer (or shorter? :O)
* Add regularization (e.g. `keras.layers.Dropout()` before the output layer)
* Do something about `_initial_layer()`?
<br>
<br>
*Feel free to comment!*
